In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sukmaadhiwijaya/welding-defect-object-detection")

print("Path to dataset files:", path)

In [ ]:
# Install YOLOv8 and necessary dependencies
!pip install ultralytics

import os
import shutil
import matplotlib.pyplot as plt
import cv2
import random
from IPython.display import Image, display
import pandas as pd
import seaborn as sns
from ultralytics import YOLO

In [ ]:
# Define paths
root_dir = path
v1_dir = os.path.join(root_dir, 'The Welding Defect Dataset/The Welding Defect Dataset')
v2_dir = os.path.join(root_dir, 'The Welding Defect Dataset - v2/The Welding Defect Dataset - v2')

In [ ]:
# Function to display images
def display_images(image_dir, num_images=9):
    images = os.listdir(image_dir)
    sample_images = random.sample(images, num_images)
    plt.figure(figsize=(15, 15))
    for i, image_name in enumerate(sample_images):
        image_path = os.path.join(image_dir, image_name)
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.subplot(3, 3, i + 1)
        plt.imshow(img_rgb)
        plt.axis('off')
    plt.show()

# Display some training images
print("Displaying some training images from version 1:")
display_images(os.path.join(v1_dir, 'train/images'))

print("Displaying some training images from version 2:")
display_images(os.path.join(v2_dir, 'train/images'))


In [ ]:
# Copy dataset to YOLOv8 directory
def copy_dataset_to_yolo(src_dir, dest_dir):
    if os.path.exists(dest_dir):
        shutil.rmtree(dest_dir)
    shutil.copytree(src_dir, dest_dir)

# Copy version 1 dataset
copy_dataset_to_yolo(v1_dir, 'yolov8/data/welding_v1')

# Copy version 2 dataset
copy_dataset_to_yolo(v2_dir, 'yolov8/data/welding_v2')

In [ ]:
# Update data.yaml paths
def update_data_yaml(data_yaml_path, base_dir):
    with open(data_yaml_path, 'r') as file:
        data_yaml = file.read()
    data_yaml = data_yaml.replace('../train/images', os.path.join(base_dir, 'train/images'))
    data_yaml = data_yaml.replace('../valid/images', os.path.join(base_dir, 'valid/images'))
    data_yaml = data_yaml.replace('../test/images', os.path.join(base_dir, 'test/images'))
    with open(data_yaml_path, 'w') as file:
        file.write(data_yaml)

# Update data.yaml for version 1
update_data_yaml('yolov8/data/welding_v1/data.yaml', '/kaggle/working/yolov8/data/welding_v1')

# Update data.yaml for version 2
update_data_yaml('yolov8/data/welding_v2/data.yaml', '/kaggle/working/yolov8/data/welding_v2')

# Print and verify updated data.yaml files
def print_data_yaml(data_yaml_path):
    with open(data_yaml_path, 'r') as file:
        print(file.read())

print("Updated data.yaml for version 1:")
print_data_yaml('yolov8/data/welding_v1/data.yaml')

print("Updated data.yaml for version 2:")
print_data_yaml('yolov8/data/welding_v2/data.yaml')


Updated data.yaml for version 1:
train: /kaggle/working/yolov8/data/welding_v1/train/images
val: /kaggle/working/yolov8/data/welding_v1/valid/images
test: /kaggle/working/yolov8/data/welding_v1/test/images

nc: 3
names: ['Bad Weld', 'Good Weld', 'Defect']


Updated data.yaml for version 2:
train: /kaggle/working/yolov8/data/welding_v2/train/images
val: /kaggle/working/yolov8/data/welding_v2/valid/images
test: /kaggle/working/yolov8/data/welding_v2/test/images

nc: 3
names: ['Bad Weld', 'Good Weld', 'Defect']




In [ ]:
# EDA and Preprocessing
def load_labels(label_dir):
    label_files = os.listdir(label_dir)
    data = []
    for label_file in label_files:
        with open(os.path.join(label_dir, label_file), 'r') as file:
            lines = file.readlines()
            for line in lines:
                class_id, x_center, y_center, width, height = map(float, line.strip().split())
                data.append([label_file, class_id, x_center, y_center, width, height])
    df = pd.DataFrame(data, columns=['file', 'class_id', 'x_center', 'y_center', 'width', 'height'])
    return df

v1_train_labels = load_labels(os.path.join(v1_dir, 'train/labels'))
v2_train_labels = load_labels(os.path.join(v2_dir, 'train/labels'))

In [ ]:
v1_train_labels.head()

,file,class_id,x_center,y_center,width,height
0,bad_weld_vid694_jpeg_jpg.rf.0f951e90c31e806acf...,0.0,0.565625,0.506250,0.117188,0.896875
1,bad_weld_vid694_jpeg_jpg.rf.0f951e90c31e806acf...,2.0,0.579688,0.691406,0.051562,0.535937
2,bad_weld_vid694_jpeg_jpg.rf.0f951e90c31e806acf...,0.0,0.732812,0.643750,0.164062,0.698438
3,SampleV2_1_mp4-10_jpg.rf.d1e56bfef0409aea47417...,2.0,0.791406,0.660156,0.196875,0.098437
4,SampleV2_1_mp4-10_jpg.rf.d1e56bfef0409aea47417...,2.0,0.563281,0.639062,0.167969,0.107813


In [ ]:
# Train YOLOv8 model with augmentation and hyperparameter tuning
# Initialize model
model_v1 = YOLO('yolov8m.pt')  # YOLOv8m (medium) model
model_v2 = YOLO('yolov8m.pt')  # YOLOv8m (medium) model

100%|██████████| 49.7M/49.7M [00:00<00:00, 167MB/s]


In [ ]:
# Train model on version 1 dataset
model_v1.train(data=os.path.join(v1_dir, "data.yaml"), epochs=50, imgsz=640)

In [ ]:
model_v1.info()

Model summary: 295 layers, 25,858,057 parameters, 0 gradients, 79.1 GFLOPs


(295, 25858057, 0, 79.0720512)

In [ ]:
model_v1.val(data=os.path.join(v2_dir, "data.yaml"),  epochs=100, imgsz=640, batch=16, name='welding_v1_aug',
               lr0=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1,
               box=0.05, cls=0.5, iou=0.2, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, translate=0.1, scale=0.5, mosaic=1.0, mixup=0.5)

Ultralytics 8.3.72 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)


val: Scanning /root/.cache/kagglehub/datasets/sukmaadhiwijaya/welding-defect-object-detection/versions/2/The Welding Defect Dataset - v2/The Welding Defect Dataset - v2/valid/labels... 283 images, 0 backgrounds, 0 corrupt: 100%|██████████| 283/283 [00:00<00:00, 664.99it/s]

val: New cache created: /root/.cache/kagglehub/datasets/sukmaadhiwijaya/welding-defect-object-detection/versions/2/The Welding Defect Dataset - v2/The Welding Defect Dataset - v2/valid/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:11<00:00,  1.58it/s]


                   all        283        802      0.648      0.647      0.625      0.365
Speed: 0.4ms preprocess, 22.9ms inference, 0.0ms loss, 3.9ms postprocess per image
Results saved to runs/detect/welding_v1_aug3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78fac6bb2590>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [ ]:
model_v1_10 = YOLO('yolov10m.pt')  # YOLOv10m (medium) model

100%|██████████| 32.1M/32.1M [00:00<00:00, 99.5MB/s]


In [ ]:

model_v1_10.train(data=os.path.join(v1_dir, "data.yaml"), epochs=100, imgsz=640, momentum=0.937,iou=0.5)

Ultralytics 8.3.73 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov10m.pt, data=/root/.cache/kagglehub/datasets/sukmaadhiwijaya/welding-defect-object-detection/versions/2/The Welding Defect Dataset/The Welding Defect Dataset/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=N

100%|██████████| 755k/755k [00:00<00:00, 20.1MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1     78720  ultralytics.nn.modules.block.SCDown          [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1    228672  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 96.0MB/s]


AMP: checks passed ✅


...

                   all        176        573      0.577      0.561      0.563      0.344



100 epochs completed in 1.054 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 33.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 33.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.73 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv10m summary (fused): 369 layers, 16,453,858 parameters, 0 gradients, 63.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:03<00:00,  1.60it/s]


                   all        176        573      0.589      0.546       0.58      0.346
              Bad Weld         87        127      0.592      0.638      0.643      0.405
             Good Weld        119        253      0.684      0.656      0.699      0.464
                Defect         90        193       0.49      0.344        0.4      0.169
Speed: 0.3ms preprocess, 12.3ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7dad16383910>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04